## Load data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB 
from numpy.core.umath_tests import inner1d
from copy import deepcopy
from sklearn import svm, datasets
from sklearn.metrics import classification_report
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
df_train = pd.read_csv('C:/Users/sq/Project/IOT/IOT-Code/iot_device_train.csv')
df_test = pd.read_csv('C:/Users/sq/Project/IOT/IOT-Code/iot_device_test.csv')

df = pd.concat([df_train,df_test],ignore_index=True)

<ipython-input-1-cb273a8676ea>:15: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Delete unique data

In [2]:
to_remove = []
for col in df.columns:
    if len(df[col].unique()) == 1:
        to_remove.append(col)
df = df.drop(to_remove, axis=1)

X = df.drop('device_category', axis=1)
y = df['device_category']
#y=pd.get_dummies(y)#Convert all categories to numeric types

In [3]:
def encodeLabel(data):
    listLable=[]
    #这里我的标签的名字全都打成了lable，我知道标签的英文是label，如果大家实在看不惯想改过来的话记得前面加载的csv文件的开头的lable也改成label
    for lable in data:
        listLable.append(lable)
    #到这里都是把lable整合到一起，下面是规格化处理
    le = LabelEncoder()
    resultLable=le.fit_transform(listLable)
    return resultLable

In [4]:
y=encodeLabel(y)
y

array([4, 4, 4, ..., 8, 8, 8], dtype=int64)

## Split

In [5]:
from sklearn import preprocessing
#normalization
input_features = preprocessing.StandardScaler().fit_transform(X)
#split
X_train, X_test, y_train, y_test = train_test_split(input_features,y, test_size=0.3, random_state=42)

## Sklearn 

In [6]:
model_ada =  AdaBoostClassifier()
model_ada.fit(X_train, y_train)
fit_pred = model_ada.predict(X_test)
 
print(classification_report(y_test, fit_pred))
#report = classification_report(y_test, fit_pred,output_dict=True)
#report= pd.DataFrame(report).transpose()

              precision    recall  f1-score   support

           0       0.69      0.82      0.75        57
           1       0.79      0.94      0.86        48
           2       0.00      0.00      0.00        48
           3       0.50      0.93      0.65        67
           4       1.00      0.14      0.24        58
           5       0.20      0.45      0.28        64
           6       0.00      0.00      0.00        57
           7       0.59      1.00      0.74        67
           8       0.86      0.71      0.78        69
           9       0.00      0.00      0.00        35

    accuracy                           0.54       570
   macro avg       0.46      0.50      0.43       570
weighted avg       0.49      0.54      0.46       570



C:\Users\sq\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sq\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sq\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Tree structure as a base classifier does not work well for integration here, therefore, we try to replace the classifier.

## Train a Adaboost classifier from scratch

### 1.Define

In [7]:
class AdaClassifier(object):
    def __init__(self, *args, **kwargs):
        if kwargs and args:
            raise ValueError(
                '''AdaBoostClassifier can only be called with the following keyword arguments：
                base_estimator,n_estimators,learning_rate,algorithm,random_state
                ''')
        allowed_keys = ['base_estimator', 'n_estimators', 'learning_rate', 'algorithm', 'random_state']
        keywords_used = kwargs.keys()
        for keyword in keywords_used:
            if keyword not in allowed_keys:
                raise ValueError(keyword + ":  Wrong keyword used --- check spelling")

        n_estimators = 50
        learning_rate = 1
        algorithm = 'SAMME.R'
        random_state = None

        if kwargs and not args:
            if 'base_estimator' in kwargs:
                base_estimator = kwargs.pop('base_estimator')
            else:
                raise ValueError('''base_estimator can not be None''')
            if 'n_estimators' in kwargs: n_estimators = kwargs.pop('n_estimators')
            if 'learning_rate' in kwargs: learning_rate = kwargs.pop('learning_rate')
            if 'algorithm' in kwargs: algorithm = kwargs.pop('algorithm')
            if 'random_state' in kwargs: random_state = kwargs.pop('random_state')

        self.base_estimator_ = base_estimator
        # self.base_estimator_ = SVC(probability=True)
        self.n_estimators_ = n_estimators
        self.learning_rate_ = learning_rate
        self.algorithm_ = algorithm
        self.random_state_ = random_state
        self.estimators_ = list()
        #self.estimators_ =base_estimator

        self.estimator_weights_ = np.zeros(self.n_estimators_)
        self.estimator_errors_ = np.ones(self.n_estimators_)


    def _samme_proba(self, estimator, n_classes, X):#model,1
        """Calculate algorithm 4, step 2, equation c) of Zhu et al [1].

        References
        ----------
        .. [1] J. Zhu, H. Zou, S. Rosset, T. Hastie, "Multi-class AdaBoost", 2009.

        """
        proba = estimator.predict_proba(X)#return probability
        #print('\n', n_classes)
        # Displace zero probabilities so the log is defined.
        # Also fix negative elements which may occur with
        # negative sample weights.
        proba[proba < np.finfo(proba.dtype).eps] = np.finfo(proba.dtype).eps
        log_proba = np.log(proba)

        return (n_classes - 1) * (log_proba - (1. / n_classes)
                                  * log_proba.sum(axis=1)[:, np.newaxis])


    def fit(self, X, y):#train
        self.n_samples = X.shape[0]
        #self.n_samples = X_train.shape[0]
        # There is hidden trouble for classes, here the classes will be sorted.
        self.classes_ = np.array(sorted(list(set(y))))
        
        self.n_classes_ = len(self.classes_)#The number of categories
        for iboost in range(self.n_estimators_):#num of base estimators
            if iboost == 0:
                sample_weight = np.ones(self.n_samples) / self.n_samples # Weight Initialization：1/N

            sample_weight, estimator_weight, estimator_error = self.boost(X,y, sample_weight)#

            # early stop
            if estimator_error == None:
                break

            # append error and weight
            self.estimator_errors_[iboost] = estimator_error
            self.estimator_weights_[iboost] = estimator_weight

            if estimator_error <= 0:
                break

        return self


    def boost(self, X, y, sample_weight):
        if self.algorithm_ == 'SAMME':
            return self.discrete_boost(X, y, sample_weight)
        elif self.algorithm_ == 'SAMME.R':
            return self.real_boost(X, y, sample_weight)

    def real_boost(self, X, y, sample_weight):
        estimator = deepcopy(self.base_estimator_)#Deep copy produces a new object estimator store base classifier
        if self.random_state_:
            estimator.set_params(random_state=1)

        estimator.fit(X, y, sample_weight=sample_weight)#train

        y_pred = estimator.predict(X)#
        incorrect = y_pred != y
        estimator_error = np.dot(incorrect, sample_weight) / np.sum(sample_weight, axis=0)

        # if worse than random guess, stop boosting
        if estimator_error >= 1.0 - 1 / self.n_classes_:#1- 1/10
            return None, None, None

        y_predict_proba = estimator.predict_proba(X)
        # repalce zero
        y_predict_proba[y_predict_proba < np.finfo(y_predict_proba.dtype).eps] = np.finfo(y_predict_proba.dtype).eps

        y_codes = np.array([-1. / (self.n_classes_ - 1), 1.])
        y_coding = y_codes.take(self.classes_ == y[:, np.newaxis])

        # for sample weight update
        intermediate_variable = (-1. * self.learning_rate_ * (((self.n_classes_ - 1) / self.n_classes_) *
                                                              inner1d(y_coding, np.log(
                                                                  y_predict_proba))))  #dot iterate for each row

        # update sample weight
        sample_weight *= np.exp(intermediate_variable)

        sample_weight_sum = np.sum(sample_weight, axis=0)
        if sample_weight_sum <= 0:
            return None, None, None

        # normalize sample weight
        sample_weight /= sample_weight_sum

        # append the estimator
        self.estimators_.append(estimator)

        return sample_weight, 1, estimator_error


    def discrete_boost(self, X, y, sample_weight):
        estimator = deepcopy(self.base_estimator_)
        if self.random_state_:
            estimator.set_params(random_state=1)

        estimator.fit(X, y, sample_weight=sample_weight)

        y_pred = estimator.predict(X)
        incorrect = y_pred != y
        estimator_error = np.dot(incorrect, sample_weight) / np.sum(sample_weight, axis=0)

        # if worse than random guess, stop boosting
        if estimator_error >= 1 - 1 / self.n_classes_:
            return None, None, None

        # update estimator_weight
        estimator_weight = self.learning_rate_ * np.log((1 - estimator_error) / estimator_error) + np.log(
            self.n_classes_ - 1)

        if estimator_weight <= 0:
            return None, None, None

        # update sample weight
        sample_weight *= np.exp(estimator_weight * incorrect)

        sample_weight_sum = np.sum(sample_weight, axis=0)
        if sample_weight_sum <= 0:
            return None, None, None

        # normalize sample weight
        sample_weight /= sample_weight_sum

        # append the estimator
        self.estimators_.append(estimator)

        return sample_weight, estimator_weight, estimator_error

    def predict(self, X):
        n_classes = self.n_classes_
        classes = self.classes_[:, np.newaxis]
        pred = None

        if self.algorithm_ == 'SAMME.R':
            # The weights are all 1. for SAMME.R
            pred = sum(self._samme_proba(estimator, X) for estimator in self.estimators_)
        else:  # self.algorithm == "SAMME"
            pred = sum((estimator.predict(X) == classes).T * w
                       for estimator, w in zip(self.estimators_,
                                               self.estimator_weights_))

        pred /= self.estimator_weights_.sum()
        if n_classes == 2:
            pred[:, 0] *= -1
            pred = pred.sum(axis=1)
            return self.classes_.take(pred > 0, axis=0)

        return self.classes_.take(np.argmax(pred, axis=1), axis=0)


    def predict_proba(self,X):
        n_classes = self.n_classes_
        if self.algorithm_ == 'SAMME.R':
            # The weights are all 1. for SAMME.R
            proba = sum(self._samme_proba(estimator, self.n_classes_, X)
                        for estimator in self.estimators_)
        else:  # self.algorithm == "SAMME"
            proba = sum(estimator.predict_proba(X) * w
                        for estimator, w in zip(self.estimators_,
                                                self.estimator_weights_))

        proba /= self.estimator_weights_.sum()
        proba = np.exp((1. / (n_classes - 1)) * proba)
        normalizer = proba.sum(axis=1)[:, np.newaxis]
        normalizer[normalizer == 0.0] = 1.0
        proba /= normalizer#归一化
        
        #pre_label=predict_label(proba)
       # print(pre_label)
        return proba#,pre_label
    '''
    def predict_label(self,proba):
        pre_label=np.atgmax(proba,axis=1)  
        return pre_label
        '''

### 2.Choose GaussianNB as the base classifier

In [8]:
model_GB=AdaClassifier(base_estimator= GaussianNB(),n_estimators=10, learning_rate=0.1, algorithm= 'SAMME.R', random_state=None)
#(probability=True),

In [9]:
model_GB

In [10]:
model_GB.fit(X_train,y_train)

In [11]:
model_GB._samme_proba(model,10,X_train)#model,1

NameError: name 'model' is not defined

In [ ]:
y_pre=model_GB.predict_proba(X_test)

In [ ]:
y_pre#output probability

In [ ]:
y_label=np.argmax(y_pre,axis=1) #Take the category with the highest probability of prediction

In [ ]:
y_label

In [ ]:
print(classification_report(y_test, y_label))

### 3.Choose SVC as the base classifier

In [ ]:
model_SVC=AdaClassifier(base_estimator= SVC(probability=True),n_estimators=10, learning_rate=0.1, algorithm= 'SAMME.R', random_state=None)

In [ ]:
model_SVC.fit(X_train,y_train)

In [ ]:
y_pre=model_SVC.predict_proba(X_test)
y_label=np.argmax(y_pre,axis=1)
print(classification_report(y_test, y_label))